<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#AB-Testing" data-toc-modified-id="AB-Testing-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>AB-Testing</a></span></li></ul></div>

# AB-Testing

![cats](images/cats.jpeg)


Imaginad que somos los cientificos de datos de la empresa de videojuegos Tactile Entertainment. Los desarrolladores del juego Cookie Cats pretenden introducir un cambio en el juego para aumentar la retencion de los jugadores. En cierto nivel del juego los jugadores se encuentran una puerta que les obliga a esperar o a pagar la app. Actualmente la puerta se encuentra en nivel 30 y se pretende pasar al nivel 40, para comprobar la retencion a 1 y 7 dias. Antes de realizar el cambio definitivo en el juego se raliza un test AB.

Los datos estan alojados en `data/cookie_cats.csv`. Nuestro grupo de control sera la version actual `gate_30` y el grupo de tratamiento sera la version `gate_40`. Debemos realizar el test para 1 dia de retencion `retention_1` y para 7 dias `retention_7`.

In [1]:
# librerias

import pandas as pd
import numpy as np

from statsmodels.stats.proportion import proportions_ztest, proportion_confint
from scipy.stats import norm, sem

import pylab as plt

In [2]:
data = pd.read_csv('data/cookie_cats.csv')

data.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True


In [23]:
data.userid.value_counts() # Cada usuario es único, no hay interferencia entre grupos de testeo

116        1
6632278    1
6658202    1
6658194    1
6658134    1
          ..
3347358    1
3347337    1
3346992    1
3346979    1
9999861    1
Name: userid, Length: 90189, dtype: int64

In [24]:
ret1 = data[['version','retention_1']] # Datos para el primer AB test

ret1

,version,retention_1
0,gate_30,False
1,gate_30,True
2,gate_40,True
3,gate_40,False
4,gate_40,True
...,...,...
90184,gate_40,True
90185,gate_40,False
90186,gate_30,True
90187,gate_40,True


In [25]:
ret7 = data[['version','retention_7']] # Datos para el segundo AB test

ret7

,version,retention_7
0,gate_30,False
1,gate_30,False
2,gate_40,False
3,gate_40,False
4,gate_40,True
...,...,...
90184,gate_40,False
90185,gate_40,False
90186,gate_30,False
90187,gate_40,False


No voy a seleccionar muestras, mejor tirar del dataset entero ya que lo tenemos. Además, como no hay ninguna hipótesis concreta de tasas de retorno que testear -sólo queremos ver si hay diferencia entre ambas estrategias- no necesitamos un tamaño de muestra específico, así que tiraremos con lo que hay.

In [26]:
# Testeo para retention_1

from statsmodels.stats.proportion import proportions_ztest, proportion_confint

# Vamos a hacerlo con una pivot table:

data1 = pd.crosstab(ret1.version, ret1.retention_1)

data1

retention_1,False,True
version,,
gate_30,24666,20034
gate_40,25370,20119


In [28]:
impresiones = [data1.iloc[0,0] + data1.iloc[0,1], data1.iloc[1,0] + data1.iloc[1,1]] # [control 30, nuevo 40]

retenciones = [data1.iloc[0,1], data1.iloc[1,1]] # [control 30, nuevo 40]

z_score, p_value = proportions_ztest(retenciones, nobs = impresiones) 

(gate_30a, gate_40a), (gate_30b, gate_40b) = proportion_confint(retenciones, 
                                                                nobs = impresiones,
                                                                alpha = 0.05)

print(f'z-score: {z_score:.2f}')

print(f'p-valor: {p_value:.3f}')

print(f'intervalo conf 95% para grupo control: [{gate_30a:.3f}, {gate_30b:.3f}]')

print(f'intervalo conf 95% para grupo tratamiento: [{gate_40a:.3f}, {gate_40b:.3f}]')

z-score: 1.78
p-valor: 0.074
intervalo conf 95% para grupo control: [0.444, 0.453]
intervalo conf 95% para grupo tratamiento: [0.438, 0.447]


Con respecto a la tasa de retención 1 día después de llegar a la puerta, el $p$-valor sale 0.074 que es mayor que $\alpha$ = 0.05... no podemos rechazar la hipótesis nula $H_0$ (aunque por poco) lo que significa que la nueva configuración del videojuego no tiene una tasa de retención significativamente diferente a la vieja.

Echemos un ojo a la retención tras 7 días a ver si es lo mismo:

In [31]:
# Testeo para retention_7

data7 = pd.crosstab(ret7.version, ret7.retention_7)

data7

retention_7,False,True
version,,
gate_30,36198,8502
gate_40,37210,8279


In [32]:
impresiones = [data7.iloc[0,0] + data7.iloc[0,1], data7.iloc[1,0] + data7.iloc[1,1]] # [control 30, nuevo 40]

retenciones = [data7.iloc[0,1], data7.iloc[1,1]] # [control 30, nuevo 40]

z_score, p_value = proportions_ztest(retenciones, nobs = impresiones) 

(gate_30a, gate_40a), (gate_30b, gate_40b) = proportion_confint(retenciones, 
                                                                nobs = impresiones,
                                                                alpha = 0.05)

print(f'z-score: {z_score:.2f}')

print(f'p-valor: {p_value:.3f}')

print(f'intervalo conf 95% para grupo control: [{gate_30a:.3f}, {gate_30b:.3f}]')

print(f'intervalo conf 95% para grupo tratamiento: [{gate_40a:.3f}, {gate_40b:.3f}]')

z-score: 3.16
p-valor: 0.002
intervalo conf 95% para grupo control: [0.187, 0.194]
intervalo conf 95% para grupo tratamiento: [0.178, 0.186]


En este caso, el $p$-valor sale 0.002 que sí es menor que $\alpha$ = 0.05. Podemos rechazar la hipótesis nula de que no hay diferencias entre poner la espera en la puerta 30 o en la puerta 40, pero no como queríamos. 

La nueva configuración del videojuego tiene una peor tasa de retención que la vieja, ya que si nos fijamos en los intervalos de confianza vemos que la tasa media del segundo grupo es menor y que además los intervalos de ambos no solapan en ningún punto.

Al 95% de confianza, podemos concluir que la estrategia antigua es mejor.